In [16]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
from scipy import stats
from statistics import mean
import math

from secret import IEX_CLOUD_API_TOKEN

In [2]:
stock_tickers = pd.read_csv('Data/sp_500_stocks.csv')
stock_tickers

,Ticker
0,MSFT
1,AAPL
2,NVDA
3,GOOG
4,GOOGL
...,...
498,IVZ
499,GNRC
500,CMA
501,XRAY


In [3]:
symbol = 'AAPL'
api_base_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
get_response = requests.get(api_base_url)
data = get_response.json()
data

{'avgTotalVolume': 73817592,
 'calculationPrice': 'close',
 'change': 0.91,
 'changePercent': 0.00531,
 'close': 172.28,
 'closeSource': 'official',
 'closeTime': 1711137600319,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 172.29,
 'delayedPriceTime': 1711137586340,
 'extendedChange': 0.02,
 'extendedChangePercent': 0.00012,
 'extendedPrice': 172.3,
 'extendedPriceTime': 1711151999309,
 'high': 173.05,
 'highSource': '15 minute delayed price',
 'highTime': 1711137599994,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 172.27,
 'iexCloseTime': 1711137598471,
 'iexLastUpdated': 1711137598471,
 'iexMarketPercent': 0.013333532321143053,
 'iexOpen': 171.8,
 'iexOpenTime': 1711114200855,
 'iexRealtimePrice': 172.27,
 'iexRealtimeSize': 3,
 'iexVolume': 948816,
 'lastTradeTime': 1711137599913,
 'latestPrice': 172.28,
 'latestSource': 'Close',
 'latestTime': 'March 22, 2024',
 'latestUpdate': 1711137600319,
 'latestVolume': 71160138,


In [4]:
price = data['latestPrice']
p_e_ratio = data['peRatio']
p_e_ratio

26.83

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

stock_ticker_sublists = list(chunks(stock_tickers['Ticker'], 100))
stock_tickers = []
for i in range(0, len(stock_ticker_sublists)):
    stock_tickers.append(','.join(stock_ticker_sublists[i]))

Dataframe = pd.DataFrame(columns = ['Ticker', 'Stock Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy'])

for stock_ticker in stock_tickers:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={stock_ticker}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for stock in stock_ticker.split(','):
        Dataframe = Dataframe.append(
            pd.Series(
                [
                    stock,
                    data[stock]['quote']['latestPrice'],
                    data[stock]['quote']['peRatio'],
                    'N/A'
                ],
                index = Dataframe.columns
            ),
            ignore_index = True
        )

Dataframe

C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\2740025539.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\2740025539.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\2740025539.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\2740025539.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\2740025539.py:16: 

,Ticker,Stock Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MSFT,428.74,38.76,N/A
1,AAPL,172.28,26.83,N/A
2,NVDA,942.89,79.04,N/A
3,GOOG,151.77,25.99,N/A
4,GOOGL,150.77,25.82,N/A
...,...,...,...,...
498,IVZ,15.71,-21.48,N/A
499,GNRC,115.91,35.45,N/A
500,CMA,51.65,8.05,N/A
501,XRAY,32.75,-52.6,N/A


In [6]:
Dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
Dataframe = Dataframe[Dataframe['Price-to-Earnings Ratio'] > 0]
Dataframe = Dataframe[:50]
Dataframe.reset_index(inplace = True)
Dataframe.drop('index', axis = 1, inplace = True)
Dataframe

,index,Ticker,Stock Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,7,BRK.B,411.6,0.01,N/A
1,476,APA,33.46,3.62,N/A
2,147,EMR,112.45,5.85,N/A
3,182,GM,43.06,5.88,N/A
4,414,UAL,46.47,5.89,N/A
5,494,FMC,62.36,5.93,N/A
6,398,EG,384.17,6.3,N/A
7,281,DAL,45.59,6.36,N/A
8,159,VLO,169.64,6.8,N/A
9,342,APTV,78.72,7.66,N/A


In [7]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        val = float(portfolio_size)
    except ValueError:
        print('Please enter an integer')
        portfolio_size = input('Enter the value of your portfolio:')

portfolio_input()

position_size = float(portfolio_size) / len(Dataframe.index)
for i in range(0, len(Dataframe.index)):
    Dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / Dataframe.loc[i, 'Stock Price'])

Dataframe

,index,Ticker,Stock Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,7,BRK.B,411.6,0.01,4
1,476,APA,33.46,3.62,59
2,147,EMR,112.45,5.85,17
3,182,GM,43.06,5.88,46
4,414,UAL,46.47,5.89,43
5,494,FMC,62.36,5.93,32
6,398,EG,384.17,6.3,5
7,281,DAL,45.59,6.36,43
8,159,VLO,169.64,6.8,11
9,342,APTV,78.72,7.66,25


In [9]:
symbol = 'AAPL'
batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_url).json()
data[symbol]['advanced-stats']

{'beta': 1.0730673872643153,
 'totalCash': 73100000000,
 'currentDebt': 133973000000,
 'revenue': 385706000000,
 'grossProfit': 173671000000,
 'totalRevenue': 385706000000,
 'EBITDA': 130109000000,
 'revenuePerShare': 24.98,
 'revenuePerEmployee': 2623850.34,
 'debtToEquity': 4.770769230769231,
 'profitMargin': 0.2616319165374664,
 'enterpriseValue': 2695267258680,
 'enterpriseValueToRevenue': 6.99,
 'priceToSales': 6.9,
 'priceToBook': 35.9,
 'forwardPERatio': 26.1237013384645,
 'pegRatio': 2.8702504473131274,
 'peHigh': 13.290389404294446,
 'peLow': 9.951190202587174,
 'week52highDate': '2023-12-14',
 'week52lowDate': '2023-03-28',
 'putCallRatio': None,
 'companyName': 'Apple Inc',
 'marketcap': 2660327258680,
 'week52high': 199.11,
 'week52low': 154.54,
 'week52highSplitAdjustOnly': 199.62,
 'week52highDateSplitAdjustOnly': '2023-12-14',
 'week52lowSplitAdjustOnly': 155.98,
 'week52lowDateSplitAdjustOnly': '2023-03-28',
 'week52change': 0.08507592319315149,
 'sharesOutstanding': 15

In [12]:

Dataframe = pd.DataFrame(columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
])

for stock in stock_tickers:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={stock}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in stock.split(','):
        try:
            ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']
        except TypeError:
            ebitda = np.NaN

        try:
            ev_to_gp = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']
        except TypeError:
            ev_to_gp = np.NaN

        Dataframe = Dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gp,
                    'N/A',
                    'N/A'
                ],
                index = Dataframe.columns
            ),
            ignore_index = True
        )
    
Dataframe

C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\1980835980.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\1980835980.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\1980835980.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\1980835980.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataframe = Dataframe.append(
C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\1980835980.py:32: 

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MSFT,428.74,N/A,38.76,N/A,13.37,N/A,14,N/A,26.977555,N/A,20.026631,N/A,N/A
1,AAPL,172.28,N/A,26.83,N/A,35.9,N/A,6.9,N/A,20.715456,N/A,15.519386,N/A,N/A
2,NVDA,942.89,N/A,79.04,N/A,54.85,N/A,38.69,N/A,67.892981,N/A,52.841922,N/A,N/A
3,GOOG,151.77,N/A,25.99,N/A,6.19,N/A,5.7,N/A,17.303619,N/A,9.506993,N/A,N/A
4,GOOGL,150.77,N/A,25.82,N/A,6.19,N/A,5.7,N/A,17.303619,N/A,9.506993,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,IVZ,15.71,N/A,-21.48,N/A,0.6666,N/A,1.23,N/A,14.534854,N/A,2.483923,N/A,N/A
499,GNRC,115.91,N/A,35.45,N/A,2.99,N/A,1.74,N/A,17.031655,N/A,6.122637,N/A,N/A
500,CMA,51.65,N/A,8.05,N/A,1.14,N/A,1.29,N/A,5.558969,N/A,1.314462,N/A,N/A
501,XRAY,32.75,N/A,-52.6,N/A,2.06,N/A,1.71,N/A,13.654693,N/A,4.110809,N/A,N/A


In [13]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    Dataframe[column].fillna(Dataframe[column].mean(), inplace = True)

Dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MSFT,428.74,N/A,38.76,N/A,13.3700,N/A,14.0000,N/A,26.977555,N/A,20.026631,N/A,N/A
1,AAPL,172.28,N/A,26.83,N/A,35.9000,N/A,6.9000,N/A,20.715456,N/A,15.519386,N/A,N/A
2,NVDA,942.89,N/A,79.04,N/A,54.8500,N/A,38.6900,N/A,67.892981,N/A,52.841922,N/A,N/A
3,GOOG,151.77,N/A,25.99,N/A,6.1900,N/A,5.7000,N/A,17.303619,N/A,9.506993,N/A,N/A
4,GOOGL,150.77,N/A,25.82,N/A,6.1900,N/A,5.7000,N/A,17.303619,N/A,9.506993,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,IVZ,15.71,N/A,-21.48,N/A,0.6666,N/A,1.2300,N/A,14.534854,N/A,2.483923,N/A,N/A
499,GNRC,115.91,N/A,35.45,N/A,2.9900,N/A,1.7400,N/A,17.031655,N/A,6.122637,N/A,N/A
500,CMA,51.65,N/A,8.05,N/A,1.1400,N/A,1.2900,N/A,5.558969,N/A,1.314462,N/A,N/A
501,XRAY,32.75,N/A,-52.60,N/A,2.0600,N/A,1.7100,N/A,13.654693,N/A,4.110809,N/A,N/A


In [14]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in Dataframe.index:
        Dataframe.loc[row, metrics[metric]] = stats.percentileofscore(Dataframe[metric], Dataframe.loc[row, metric])/100

Dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MSFT,428.74,N/A,38.76,0.795229,13.3700,0.894632,14.0000,0.968191,26.977555,0.862823,20.026631,0.946322,N/A
1,AAPL,172.28,N/A,26.83,0.560636,35.9000,0.972167,6.9000,0.833002,20.715456,0.737575,15.519386,0.900596,N/A
2,NVDA,942.89,N/A,79.04,0.95825,54.8500,0.980119,38.6900,1.0,67.892981,0.980119,52.841922,1.0,N/A
3,GOOG,151.77,N/A,25.99,0.540755,6.1900,0.724652,5.7000,0.764414,17.303619,0.599404,9.506993,0.653082,N/A
4,GOOGL,150.77,N/A,25.82,0.536779,6.1900,0.724652,5.7000,0.764414,17.303619,0.599404,9.506993,0.653082,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,IVZ,15.71,N/A,-21.48,0.031809,0.6666,0.073559,1.2300,0.178926,14.534854,0.481113,2.483923,0.159046,N/A
499,GNRC,115.91,N/A,35.45,0.754473,2.9900,0.489066,1.7400,0.302187,17.031655,0.586481,6.122637,0.405567,N/A
500,CMA,51.65,N/A,8.05,0.093439,1.1400,0.110338,1.2900,0.192843,5.558969,0.045726,1.314462,0.063618,N/A
501,XRAY,32.75,N/A,-52.60,0.013917,2.0600,0.302187,1.7100,0.296223,13.654693,0.441352,4.110809,0.254473,N/A


In [17]:
for row in Dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(Dataframe.loc[row, metrics[metric]])
    Dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

Dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MSFT,428.74,N/A,38.76,0.795229,13.3700,0.894632,14.0000,0.968191,26.977555,0.862823,20.026631,0.946322,0.893439
1,AAPL,172.28,N/A,26.83,0.560636,35.9000,0.972167,6.9000,0.833002,20.715456,0.737575,15.519386,0.900596,0.800795
2,NVDA,942.89,N/A,79.04,0.95825,54.8500,0.980119,38.6900,1.0,67.892981,0.980119,52.841922,1.0,0.983698
3,GOOG,151.77,N/A,25.99,0.540755,6.1900,0.724652,5.7000,0.764414,17.303619,0.599404,9.506993,0.653082,0.656461
4,GOOGL,150.77,N/A,25.82,0.536779,6.1900,0.724652,5.7000,0.764414,17.303619,0.599404,9.506993,0.653082,0.655666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,IVZ,15.71,N/A,-21.48,0.031809,0.6666,0.073559,1.2300,0.178926,14.534854,0.481113,2.483923,0.159046,0.184891
499,GNRC,115.91,N/A,35.45,0.754473,2.9900,0.489066,1.7400,0.302187,17.031655,0.586481,6.122637,0.405567,0.507555
500,CMA,51.65,N/A,8.05,0.093439,1.1400,0.110338,1.2900,0.192843,5.558969,0.045726,1.314462,0.063618,0.101193
501,XRAY,32.75,N/A,-52.60,0.013917,2.0600,0.302187,1.7100,0.296223,13.654693,0.441352,4.110809,0.254473,0.26163


In [20]:
Dataframe.sort_values('RV Score', ascending = True, inplace = True)
Dataframe = Dataframe[:50]
Dataframe.reset_index(inplace = True)
Dataframe.drop('index', axis = 1, inplace = True)
Dataframe

C:\Users\gupta\AppData\Local\Temp\ipykernel_15068\1920680364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataframe.sort_values('RV Score', ascending = True, inplace = True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MSI,348.56,5,35.10,0.745527,79.97,0.99006,5.80,0.780318,24.386051,0.829026,12.516864,0.805169,0.83002
1,UBER,80.23,24,88.94,0.964215,13.27,0.887674,4.00,0.626243,77.476270,0.984095,10.353514,0.689861,0.830417
2,FAST,78.1,25,38.66,0.793241,13.35,0.892644,6.08,0.793241,26.219272,0.854871,13.334350,0.833002,0.8334
3,SPGI,419.13,4,50.93,0.874751,3.92,0.571571,10.74,0.930417,27.588812,0.868787,17.277675,0.926441,0.834394
4,FDS,447.46,4,36.20,0.763419,10.02,0.845924,8.03,0.868787,22.418220,0.787276,16.131243,0.912525,0.835586
5,IT,479.34,4,43.26,0.83499,54.93,0.982107,6.33,0.804175,29.587303,0.900596,9.623014,0.662028,0.836779
6,DAY,69.17,28,200.09,0.99006,4.49,0.606362,7.11,0.841948,57.066799,0.95825,12.356643,0.799205,0.839165
7,CPRT,57.41,34,30.87,0.653082,8.13,0.795229,13.60,0.964215,30.000249,0.904573,15.230862,0.88668,0.840755
8,ZTS,168.9,11,33.31,0.717694,15.48,0.906561,9.05,0.902584,25.148926,0.842942,13.682058,0.846918,0.84334
9,EW,93.79,21,40.78,0.821074,8.48,0.813121,9.39,0.910537,29.553838,0.898608,11.967227,0.7833,0.845328


In [21]:
portfolio_input()

position_size = float(portfolio_size) / len(Dataframe.index)
for i in range(0, len(Dataframe.index)):
    Dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / Dataframe.loc[i, 'Price'])

Dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MSI,348.56,5,35.10,0.745527,79.97,0.99006,5.80,0.780318,24.386051,0.829026,12.516864,0.805169,0.83002
1,UBER,80.23,24,88.94,0.964215,13.27,0.887674,4.00,0.626243,77.476270,0.984095,10.353514,0.689861,0.830417
2,FAST,78.1,25,38.66,0.793241,13.35,0.892644,6.08,0.793241,26.219272,0.854871,13.334350,0.833002,0.8334
3,SPGI,419.13,4,50.93,0.874751,3.92,0.571571,10.74,0.930417,27.588812,0.868787,17.277675,0.926441,0.834394
4,FDS,447.46,4,36.20,0.763419,10.02,0.845924,8.03,0.868787,22.418220,0.787276,16.131243,0.912525,0.835586
5,IT,479.34,4,43.26,0.83499,54.93,0.982107,6.33,0.804175,29.587303,0.900596,9.623014,0.662028,0.836779
6,DAY,69.17,28,200.09,0.99006,4.49,0.606362,7.11,0.841948,57.066799,0.95825,12.356643,0.799205,0.839165
7,CPRT,57.41,34,30.87,0.653082,8.13,0.795229,13.60,0.964215,30.000249,0.904573,15.230862,0.88668,0.840755
8,ZTS,168.9,11,33.31,0.717694,15.48,0.906561,9.05,0.902584,25.148926,0.842942,13.682058,0.846918,0.84334
9,EW,93.79,21,40.78,0.821074,8.48,0.813121,9.39,0.910537,29.553838,0.898608,11.967227,0.7833,0.845328


In [22]:
writer = pd.ExcelWriter('Value_Strategy.xlsx', engine = 'xlsxwriter')
Dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['Price-to-Earnings Ratio', float_format],
    'E': ['PE Percentile', percent_format],
    'F': ['Price-to-Book Ratio', float_format],
    'G': ['PB Percentile', percent_format],
    'H': ['Price-to-Sales Ratio', float_format],
    'I': ['PS Percentile', percent_format],
    'J': ['EV/EBITDA', float_format],
    'K': ['EV/EBITDA Percentile', percent_format],
    'L': ['EV/GP', float_format],
    'M': ['EV/GP Percentile', percent_format],
    'N': ['RV Score', percent_format]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_format)

writer.save()